In [1]:
import pandas as pd
import os


ticker = "SSNLF"

read_csv = f"{ticker}_daily_data.csv"

technical_analysis_csv = f"{ticker}_technical_analysis.csv"
summary_csv = f"{ticker}_summary_table.csv"

# Load the CSV file
df = pd.read_csv(f"{read_csv}")

# Rename "Unnamed: 0" to "Date" and parse it as a datetime column
df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Sort data by date (oldest to newest)
df = df.sort_values(by="Date").reset_index(drop=True)

In [2]:

import pandas as pd

def calculate_true_range(df):
    """Calculate True Range for ATR."""
    df['High-Low'] = df['high'] - df['low']
    df['High-Close'] = abs(df['high'] - df['close'].shift(1))
    df['Low-Close'] = abs(df['low'] - df['close'].shift(1))
    df['True Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df
# Volatility signal
def calculate_atr(df, period=14):
    """Calculate Average True Range (ATR) using Wilder's smoothing."""
    df = calculate_true_range(df)
    df['ATR'] = df['True Range'].ewm(alpha=1/period, adjust=False, min_periods=1).mean()
    return df
# Stock trend discovery
def calculate_adx(df, period=14):
    """Calculate Average Directional Index (ADX)."""
    df = calculate_atr(df, period)  # Ensure ATR uses Wilder's smoothing

    # Calculate +DM and -DM
    df['+DM'] = df['high'] - df['high'].shift(1)  # Up move
    df['-DM'] = df['low'].shift(1) - df['low']    # Down move (previous_low - current_low)

    # Set negative DM values to 0
    df['+DM'] = df['+DM'].clip(lower=0)
    df['-DM'] = df['-DM'].clip(lower=0)

    # Zero out the smaller DM
    mask = df['+DM'] > df['-DM']
    df.loc[~mask, '+DM'] = 0
    df.loc[mask, '-DM'] = 0

    # Smooth +DM and -DM using Wilder's method
    df['Smoothed+DM'] = df['+DM'].ewm(alpha=1/period, adjust=False).mean()
    df['Smoothed-DM'] = df['-DM'].ewm(alpha=1/period, adjust=False).mean()

    # Calculate +DI and -DI
    df['+DI'] = (df['Smoothed+DM'] / df['ATR']) * 100
    df['-DI'] = (df['Smoothed-DM'] / df['ATR']) * 100

    # Calculate DX and ADX (smoothed DX)
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].ewm(alpha=1/period, adjust=False).mean()

    return df
# Volume weights
def calculate_obv(df):
    """Calculate On-Balance Volume (OBV)."""
    df['OBV'] = (df['volume'] * ((df['close'].diff() > 0) * 2 - 1)).cumsum()
    return df
# Buy&Sell signals
def calculate_rsi(df, period=14):
    """Calculate Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['RSI'] = df['RSI'].fillna(50)  # Default initial RSI to 50
    return df
# Buy&Sell signals
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """Calculate Moving Average Convergence Divergence (MACD)."""
    df['EMA_12'] = df['close'].ewm(span=short_window, adjust=False, min_periods=1).mean()
    df['EMA_26'] = df['close'].ewm(span=long_window, adjust=False, min_periods=1).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False, min_periods=1).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    return df

def calculate_technical_indicators(df):
    """Compute technical indicators including ADX, ATR, OBV, RSI, and MACD."""
    df = df.copy()
    df = calculate_atr(df, period=14)
    df = calculate_adx(df, period=14)
    df = calculate_obv(df)
    df = calculate_rsi(df, period=14)
    df = calculate_macd(df, short_window=12, long_window=26, signal_window=9)
    return df


In [3]:
# Apply the technical indicators to the dataset
df = calculate_technical_indicators(df)

# Generate Summary Table
summary_df = df[["Date", "close", "ATR", "ADX", "OBV", "RSI", "MACD", "Signal_Line", "MACD_Histogram"]].tail(10)

# Save results to CSV
df.to_csv(f"{technical_analysis_csv}", index=False)
summary_df.to_csv(f"{summary_csv}", index=False)

# Display the summary table in a readable format
print(summary_df.to_string(index=False))

      Date   close      ATR       ADX     OBV  RSI      MACD  Signal_Line  MACD_Histogram
2025-03-21 40.5999 0.000005 99.999997 -264991 50.0 -0.000042    -0.000061        0.000020
2025-03-24 40.5999 0.000005 99.999997 -264991 50.0 -0.000039    -0.000057        0.000018
2025-03-25 40.5999 0.000004 99.999997 -264991 50.0 -0.000036    -0.000052        0.000017
2025-03-26 40.5999 0.000004 99.999997 -264991 50.0 -0.000033    -0.000049        0.000016
2025-03-27 40.5999 0.000004 99.999997 -264991 50.0 -0.000031    -0.000045        0.000014
2025-03-28 40.5999 0.000004 99.999997 -264991 50.0 -0.000028    -0.000042        0.000013
2025-03-31 40.5999 0.000003 99.999997 -264991 50.0 -0.000026    -0.000039        0.000012
2025-04-01 40.5999 0.000003 99.999997 -264991 50.0 -0.000024    -0.000036        0.000011
2025-04-02 40.5999 0.000003 99.999997 -264991 50.0 -0.000022    -0.000033        0.000011
2025-04-03 40.5999 0.000003 99.999997 -264991 50.0 -0.000021    -0.000031        0.000010
